# Used Libraries

In [1]:
# Main libraries 
from sklearn.preprocessing import LabelEncoder
from fastapi.encoders import jsonable_encoder
from pydantic import BaseModel
from fastapi import FastAPI
from enum import Enum
import pandas as pd
import nest_asyncio
import uvicorn
import os
import io

# Our files
from data_shuffling_split import *
from features_extraction import *
from data_preprocess import *
from ml_modeling import *
from configs import *

# Note !

Based on what we got from **Compare ML Models**, we end up by decide of using *Abo Bakr Word2vec model*, alongside with AdaBoost Classifier.

**We need to map the label as it numbers into corresponding class delicate do we use LabelEncoder as before**

In [2]:
number_of_features = 100
max_len_str  = 64
word2vec_path = "bakrianoo_unigram_cbow_model/full_uni_cbow_100_twitter.mdl"
word_to_vec_model = load_word2vec_model("models/word2vec/" + word2vec_path)

strat_test_set = pd.read_csv("dataset/test/strat_test_set.csv")
l_encoder = LabelEncoder()
l_encoder.fit(list(strat_test_set["dialect"]))
l_encoder.classes_

array(['AE', 'BH', 'DZ', 'EG', 'IQ', 'JO', 'KW', 'LB', 'LY', 'MA', 'OM',
       'PL', 'QA', 'SA', 'SD', 'SY', 'TN', 'YE'], dtype='<U2')

In [3]:
def classify_text(text, choosed_model, word_to_vec_model=word_to_vec_model, 
                  max_len_str=max_len_str, l_encoder=l_encoder):
    '''
    The function used to predict the tweets requested by the API.
    
    Argumen
        text              : string, the text we need to classify.
        word_to_vec_model : word2vec object, our word embedding model.
        max_len_str       : integer, The maximum number of tokens represent each text.
        l_encoder         : object, the encode of each class into some label to retrieve the class from.
    Return
        classifed_text    : dict, the returned classified text into the api request.
    '''
    # First process the text requested
    text_cleaned = clean_text(text)
    # Second tokenize that text to get the representation of each token
    tokenized_text = tokenize_using_nltk_TreebankWordTokenizer([text_cleaned])
    # Retrieve the whole text representation
    text_features = text_to_matrix_using_word2vec(word_to_vec_model, tokenized_text, max_len_str)
    
    # No prediction yet
    predicted = ''
    # Prediction with chosen ML Model
    if choosed_model == "Machine Learning Model":
        model_path    = "bakr/AdaBoostClassifier__f1_0.325_ml.sav"
        cls_model     = pickle_load_model("models/ml_models/" + model_path)
        
        # it return a list with one index to retrieve it directly
        predicted = cls_model.predict(text_features)
    elif choosed_model == "Deep Learning Model":
        predicted = cls_model.predict(text_features)
        
    
    pred_result = l_encoder.inverse_transform(predicted)
    # Return dictionary with request text and the predicted class
    classifed_text = {
        'query': text,
        'predicted_class': str(pred_result[0])
    }
    return classifed_text

In [4]:
# Assign an instance of the FastAPI class to the variable "app" to interact with the api.
app = FastAPI(title='Deploying a ML & DL Model with FastAPI')

# List available models using Enum
class Model(str, Enum):
    """The class to choose from the models we have, Enum represent fixed value that can not change"""
    ML_Model = "Machine Learning Model"
    DL_Model = "Deep Learning Model"


class Text(BaseModel):
    """
    The class used to get the text from the user, 
    """
    user_attention: str = """Put your Arabic text in empty value of key "query" below to predict it and choose which model architecture you need."""
    query         : str = ''
        

@app.get("/")
def home():
    # Once you go to this link you will see the get and post method below to trying out
    return "Congratulations! Your API is working as expected. Now head over to http://localhost:8000/docs."


# This endpoint handles all the logic necessary for the object detection to work.
# It requires the desired model and the dictionary of tweet and default class as we give default values to us
# In the api you can try other tweet from some_tweets below
@app.post("/predict") 
def prediction(model: Model, text_dict: Text):
    
    # Encode the retrived request data 
    text_dict      = jsonable_encoder(text_dict)
    # Run our model
    try:
        classifed_text = classify_text(text_dict['query'], model)
    except:
        classifed_text = {
        'Error': "Please Provide us with Arabic text in empty value of key 'query' in the Request body, Thanks ^^."
        }
    return classifed_text

In [ ]:
# Allows the server to be run in this interactive environment
nest_asyncio.apply()

# Host depends on the setup you selected
host = "0.0.0.0"

# uvicorn is fast Asynchronous Server Gateway Interface (ASGI) uvicorn handles the serving
# Spin up the server!    
uvicorn.run(app, host=host, port=8000)

INFO:     Started server process [13184]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:42866 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:42866 - "GET /openapi.json HTTP/1.1" 200 OK


[[]]
(1, 64)
INFO:     127.0.0.1:42866 - "POST /predict?model=Machine%20Learning%20Model HTTP/1.1" 200 OK
شكرا ده شىء جميل جدا
شكرا ده شيء جميل جدا
[['شكرا', 'ده', 'شيء', 'جميل', 'جدا']]
(1, 64, 100)
(1, 6400)
[ 2.857  -1.712   2.434   0.128   0.3936 -2.193   0.3833  0.055   1.605
 -4.23   -1.247  -4.336   1.614  -0.772  -1.229  -0.1295 -1.25    0.635
  3.434  -2.48   -1.474  -2.334  -4.305   0.286  -0.6567 -0.7783 -1.347
 -1.537   2.867  -1.22   -1.742   0.4807  1.909  -3.543  -1.9795 -0.507
 -1.003   6.83   -0.4705 -1.908  -4.246   3.383   0.3538  1.767  -0.9907
  1.106   2.25    1.004   0.3086 -1.436 ]
['EG']
INFO:     127.0.0.1:42868 - "POST /predict?model=Machine%20Learning%20Model HTTP/1.1" 200 OK


/home/abdelrahman/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LinearSVC from version 1.0.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/abdelrahman/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 1.0.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


هذا الحساب أكثر روعة وأنت متصل على شبكتنا.
هذا الحساب اكثر روعه وانت متصل علي شبكتنا . 
[['هذا', 'الحساب', 'اكثر', 'روعه', 'وانت', 'متصل', 'علي', 'شبكتنا', '.']]
(1, 64, 100)
(1, 6400)
[ 0.6406  -1.85     0.01985 -0.8535   0.3623  -2.416   -1.094   -2.168
  2.006   -2.81     1.923    0.06726  2.766   -3.68    -1.073   -1.608
 -0.6113   1.991    2.984    2.396   -2.871    0.1931   0.777    3.166
  3.594    1.499   -0.5244   2.85     1.408   -0.753    0.717   -2.047
 -0.5664   3.615   -0.06015  3.229   -2.357    4.137    1.76    -3.941
 -3.717   -0.2527  -0.2693  -2.22     1.606    3.387   -0.11206  1.086
  2.904   -1.972  ]
['KW']
INFO:     127.0.0.1:42910 - "POST /predict?model=Machine%20Learning%20Model HTTP/1.1" 200 OK


/home/abdelrahman/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LinearSVC from version 1.0.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/abdelrahman/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 1.0.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
